# ORGANELLE CONTACT ANALYSIS

-------

## OBJECTIVE:
In this notebook, the logic for determining locations of contacts between 2 or more organelles is outlined. Additionally, the logic for quantifying organelle contact composition (how much of each contact is present) and morphology (contact site size and shape) is outlined as well. 

## SUMMARY OF WORKFLOW STEPS:
 - Inputs
    - Get and Load image and its segmentations
    - Save segmentations to a dictionary
    - Create overlap regions dictionary
 - Run Analysis 
    - Regionprops
    - Distribution

## IMPORTS:

In [ ]:
import warnings
import numpy as np
from typing import Any, List, Union
import pandas as pd
from IPython.display import display
from pathlib import Path
import os, sys

from skimage.measure import regionprops_table, label
from skimage.segmentation import watershed

from infer_subc.core.file_io import read_czi_image, read_tiff_image
from infer_subc.core.img import apply_mask
from infer_subc.utils.batch import list_image_files, find_segmentation_tiff_files
from infer_subc.utils.stats import surface_area_from_props, get_XY_distribution, get_Z_distribution
from infer_subc.utils.stats_helpers import inkeys

from infer_subc.constants import (TEST_IMG_N,
                                  NUC_CH ,
                                  LYSO_CH ,
                                  MITO_CH ,
                                  GOLGI_CH ,
                                  PEROX_CH ,
                                  ER_CH ,
                                  LD_CH ,
                                  RESIDUAL_CH )

#For Convexhull Errors
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", RuntimeWarning)

## INPUTS:
Load image and segmentations and save segmentations to a dictionary. Using these segmentations, overlaps between 2 or more segmentations are performed and added to their own dictionary.

### User Inputs

##### Image Path
Here, the user should edit any of the values to correctly access the folder containing the __RAW__ image files. The naming of this file will then be used later to collect the corresponding segmentation files.

In [ ]:
test_img_n = 0

data_root_path = Path(os.path.expanduser("~")) / "Documents/Python Scripts/Infer-subc-2D"

in_data_path = data_root_path / "raw/shannon"
seg_path = data_root_path/"out"
seg_suffix = "-"
im_type = ".tiff"

img_file_list = list_image_files(in_data_path,im_type)
test_img_name = img_file_list[test_img_n]

out_data_path = data_root_path / "out"
if not Path.exists(out_data_path):
    Path.mkdir(out_data_path)
    print(f"making {out_data_path}")

In [ ]:
img_data,meta_dict = read_czi_image(test_img_name)

channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

##### Function Inputs
These are inputs that will be entered into the main function by the user. These should be changed to properly represent the user's image.

In [ ]:
org_names = ['LD', 'ER', 'golgi', 'lyso', 'mito', 'perox']
masks_file_name= ['nuc', 'cell']
mask = 'cell'
splitter = "_"
include_contact_dist = True
dist_centering_obj = 'nuc'
dist_center_on = False
dist_keep_center_as_bin = True
dist_num_bins = 5
dist_zernike_degrees=None

### Collecting All Organelle Segmentations and Region/Mask Segmentations
These inputs are NOT meant to be changed. These will be used by the program to collect the segmentations based off of the inputs from above and used furtherdown in the pipeline.

In [ ]:
org_segs = [read_tiff_image(find_segmentation_tiff_files(test_img_name, (org_names + masks_file_name), seg_path, seg_suffix)[org]) for org in org_names]
region_segs=[read_tiff_image(find_segmentation_tiff_files(test_img_name, (org_names + masks_file_name), seg_path, seg_suffix)[masks_file_name[0]]), read_tiff_image(find_segmentation_tiff_files(test_img_name, (org_names + masks_file_name), seg_path, seg_suffix)[masks_file_name[1]])]
mask = region_segs[masks_file_name.index(mask)]
center_obj = region_segs[masks_file_name.index(dist_centering_obj)]


### Make Dictionaries
These inputs are NOT meant to be changed and are not user facing.

Here we will convert the input list of __organelle segmentations__ and the input list of __organelle names__ into two dictionaries--one in a ***binary*** format and the other in a ***labeled*** format. 

The ***binary*** format treats the locations of the organelles in the image as 1s and the space without the organelle is assigned 0s.

The ***labeled*** format treats the locations of the organelles in the image as positive integers with each different organelle within the organelle type having a different integer value assigned to it; meanwhile, the space without the organelle is assigned 0s. 

In [ ]:
orgs_labeled = {}                                                       #Initialize dictionary
for idx, name in enumerate(org_names):                                  #Loop across each organelle name
    if name == 'ER':                                                    #Proceed only for ER
        orgs_labeled[name]=(org_segs[idx]>0).astype(np.uint16)          #Ensures ER is labeled only as one object & sets it as key for its object segmentation
    else:                                                               #Proceed for other organelles
        orgs_labeled[name]=org_segs[idx]                                #Set the organelle name as the key for the corresponding object segmentation

### Make 2-Way Contacts 
Here we are creating a dictionary of all the pairwise contacts between organelles. The method for creating pairwise contacts like this will be expanded upon to make nth order contacts. These inputs are NOT meant to be changed and are not user facing.

In [ ]:
contacts = {}                                                       #Initializes 2-Way Contacts dictionary
for a in (orgs_labeled.keys()):                                      #Iterates across all labeled organelle images (a)
    for b in (orgs_labeled.keys()):                                  #Iterates across all labeled organelle images (b)
        b_a=b+splitter+a                                            #Creates string to check for found a & b contact
        a_b=a+splitter+b                                            #Creates new potential key for a & b contact
        if ((a!=b) and not                                          #Ensures a is not the same as b
            (b_a in contacts.keys()) and                            #Ensures no contacts of a & b are already found
            np.any((orgs_labeled[a]*orgs_labeled[b]))):               #Ensures contact is between a & b
            contacts[a_b]=(label(orgs_labeled[a]*orgs_labeled[b]))    #Assigns contact of a & b to dictionary


### Make n-Way Contacts
To make n-Way contacts, there are 4 steps that are repeated the maximum number of times for the total number of contacts possible in an image.

Step 1: Finding nLO Contacts

When finding nLO contacts, the program searches for the number of organelles in the lower order contacts and selectively chooses only the ones with n-1 organelles where n is the nth order contact number.

Step 2: Making the n-Way Contact

More complex than making 2-Way contacts, making an n-Way contact requires more parameters to ensure that the contact has not already been made. While the idea of how the parameters work is the same between the 2-Way contacts and the n-Way contacts, the way that they are specified are slightly different.

Step 3: Adding n-Way Contacts to ***contacts*** Dictionary

In this step, the n-Way contacts are added to the contacts dictionary where they can be viewed as lower order contacts to the next contact order and/or used in the analysis of the contact sites.

Step 4: Adding non-redundant contacts to ***HO_conts*** Dictionary

In this step, we iterate across each nLO contact and watershed the nO contacts into them to find all the areas where a nLO contact is involved in a HO contact. All nLO contacts involved in a higher order contact are removed from a secondary dictionary of the contacts which only contains non-redundant contacts (contacts present in the highest order).
    
> nLO - This refers to the next lower order contact of the current nth order contact.

> nO - This refers to the current nth order contact.

> HO - This refers to "higher order" contacts and can be used in reference to the nLO contact, in which case an example of a higher order contact would be the nO contact and any order above it.

These inputs are NOT meant to be changed and are not user facing.

In [ ]:
iterated={}                            #Iterated Dictionary
contact={}                             #Contact Dictionary
HO_conts={}
for n in (range(len(org_names)-1)): 
    iterated.clear()                   #Clears iterated dictionary
    contact.clear()                    #Clears contact dictionary
    num=n+3                            #Number of contacts in lower order number

##########################
# Finding nLO Contacts
##########################
    for key in contacts:                                    #Iterates over every key in contacts
        if (len(key.split(splitter)) == (num-1)):           #Selects for last set of contacts
            iterated[key]=contacts[key]                     #Adds each key from last set of contacts to iterated

###########################
# Making the n-Way Contact
###########################
    for c in iterated:                                                                  #Iterates through preexisting dictionary of contact sites
        for d in orgs_labeled:                                                           #Iterates through labeled organelles list (b)
            if(np.any((iterated[c]*orgs_labeled[d])>0) and not
               ((d+splitter in c)or(splitter+d in c)or(splitter+d+splitter in c)) and   #Proceeds if organelle is not already in previous contact set
               (not inkeys(contact,(c+splitter+d),splitter=splitter))):                 #Proceeds if contact between organelle and previous contact set is not already made  
                contact[(c+splitter+d)]=label((iterated[c]*orgs_labeled[d])>0)           #Adds new binary contact

################################################
# Adding n-Way contacts to contacts Dictionary
################################################
    for key in contact:              #Iterates across the new contact sites 
        contacts[key]=(contact[key])   #Labels & assigns higher order contact to 

################################################
# Dictionary of Non-Redundant Contacts
################################################
    while len(iterated) != 0:
        key = list(iterated.keys())[0]
        ara = iterated[key]>0
        HO_conts[key] = ara
        for ki in contact.keys():
            if (set(ki.split(splitter)).issuperset(key.split(splitter))):
                HO_conts[key] = HO_conts[key]*(np.invert(watershed(image=(np.invert(ara)),                             #Watershed with the map of the nLO image
                                                                          markers=contact[ki],                       #Watershed with the markers of the nO's superstr
                                                                          mask=ara,                                  #Watershed unable to proceed beyond the nLO image
                                                                          connectivity=np.ones((3, 3, 3), bool))>0)) #Watershed with 3D connectivity
        del iterated[key]

## RUN ANALYSIS:
Here the data from the above inputs are analyzed to find everything we can about the contacts from regionprops measurements to the original organelles and their numbers' involved in the contacts to even the distribution of the contacts.

### Choose an n-Way Contact
Due to this function being designed around only functioning for a single contact site, users should input data the desired n-Way contact to analyze here. In the final take of this function, all contacts will be analyzed.

In [ ]:
orgs = "ER_perox"

### Get the Contact Metrics for Chosen n-Way Contact
This will allow us to analyze and obtain numerical data from the image and the contacts between organelles present in it. This is run in a loop across each organelle in practice, but here we are only running it across a single organelle to help break down the concepts of what is happening. 

In [ ]:
site = contacts[orgs]
para = HO_conts[orgs]
dist_tabs =[]
labels = label(apply_mask(site, mask))    #Isolate to only contact sites found within the cell of interest
para_labels = apply_mask((para>0), mask).astype("int") * labels #copy labels found in labels to para_labels

##### Create List of Regionprops Measurements And Run Regionprops
Here, a table is created using the regionprops measurements for the contact site.

In [ ]:
##########################################
## CREATE LIST OF REGIONPROPS MEASUREMENTS
##########################################
# start with LABEL
properties = ["label"]

# add position
properties = properties + ["centroid", "bbox"]

# add area
properties = properties + ["area", "equivalent_diameter"] # "num_pixels", 

# add shape measurements
properties = properties + ["extent", "euler_number", "solidity", "axis_major_length", "slice"] # "feret_diameter_max", "axis_minor_length", 

##################
## RUN REGIONPROPS
##################
props = regionprops_table(labels, intensity_image=None, properties=properties, extra_properties=None, spacing=scale)

##### Add Surface Area Columns to Data Table
This function adds the surface area of the contact. Viewing the organelles as circles in a venn-diagram and the contact as the overlapping area, the surface area is the part of each circle's surface that is inside of the other circle.

In [ ]:
surface_area_tab = pd.DataFrame(surface_area_from_props(labels, props, scale))

##### Determine Organelles Involved in Contact and Organelle's Labels
This section of code enables the determination of which organelle label of each organelle involved in the contact is present in the contact. For example, if peroxisome # 6 was involved in a contact with the ER, the contact would be listed as "ER_perox" and the label for the contact would be listed as 1_6. 

Additionally, this section of code also determines whether or not the contact is involved in a higher order contact, and lists it as redundant if it is. This means, that any contact that is of the highest order is listed as non-redundant. This does not remove redundant contacts from analysis--it only provides additional context to the data.

In [ ]:
cont_inv = []                                               #initializes a variable to be used for creating the values for the contacts in the dictionary
                                                                  #   This variable is a list of the labels of each organelle involved in one contact site between those organelles
involved = orgs.split(splitter)                             #creates list of all involved organelles in the contact
indexes = dict.fromkeys(involved, [])                       #A dictionary of indexes of site involved in the contact
indexes[orgs] = []                                           #   str = "contact" or "organelle", may have multiple different organelles
                                                                  #   list = contact or organelle number in image corresponding to the same contact in the other keys
                                                                #   a 2-way contact will have 3 keys, a 3-way contact will have 4 keys, etc
redundancy = []
for index, l in enumerate(props["label"]):
    cont_inv.clear()                                        #clears cont_inv variable of any labels from past contact site for new contact site
    present = para_labels[props["slice"][index]]            #examines contact site to find if it is present in a higher order contact or not
    present = present==l
    redundant = not np.any(present)
    redundancy.append(redundant)
    for org in involved:                                    #iterates across list of involved organelles
        volume = labels[props["slice"][index]]
        lorg = orgs_labeled[org][props["slice"][index]]
        volume = volume==l
        lorg = lorg[volume]
        all_inv = np.unique(lorg[lorg>0]).tolist()
        if len(all_inv) != 1:                               #ensures only one label is involved in the contact site
            print(f"we have an error.  as-> {all_inv}")     #informs the console of any errors and the reasoing for it
        indexes[org].append(f"{all_inv[0]}")                #adds the label of the organelle involved in the contact to the organelle's key's list
        cont_inv.append(f"{all_inv[0]}")                    #adds the label of the organelle involved in the contact to the list of involved organelle labels
    indexes[orgs].append("_".join(cont_inv))                #adds the combination of all the organelle's labels involved in the contact to the contact key's list

##### Combine the Datatables
Here, all of the individual datatables created to examine the contacts data present are combined into one data table for export.

In [ ]:
props_table = pd.DataFrame(props)
props_table.drop(columns=['slice', 'label'], inplace=True)
props_table.insert(0, 'label',value=indexes[orgs])
props_table.insert(0, "object", orgs)
props_table.rename(columns={"area": "volume"}, inplace=True)
props_table.insert(11, "surface_area", surface_area_tab)
props_table.insert(13, "SA_to_volume_ratio", 
props_table["surface_area"].div(props_table["volume"]))
if scale is not None:
    round_scale = (round(scale[0], 4), round(scale[1], 4), round(scale[2], 4))
    props_table.insert(loc=2, column="scale", value=f"{round_scale}")
else: 
    props_table.insert(loc=2, column="scale", value=f"{tuple(np.ones(labels.ndim))}") 
props_table["redundant"] = redundancy

Here, we can view the results of the analysis performed for the contacts thus far.

In [ ]:
display(props_table)

##### Calculate Distribution of Measurements
This section of code performs analysis on the distribution measurements of the contacts. It is entirely optional and whether it is performed or not is determined in the user inputs earlier on in this notebook. 

This section of code will additional combine the distribution measurements into a separate table for export.

In [ ]:
if include_contact_dist:
    XY_contact_dist, XY_bins, XY_wedges = get_XY_distribution(mask=mask, 
                                                              obj=site,
                                                              obj_name=orgs,
                                                              centering_obj=center_obj,
                                                              scale=scale,
                                                              center_on=dist_center_on,
                                                              keep_center_as_bin=dist_keep_center_as_bin,
                                                              num_bins=dist_num_bins,
                                                              zernike_degrees=dist_zernike_degrees)
          
    Z_contact_dist = get_Z_distribution(mask=mask,
                                        obj=site,
                                        obj_name=orgs,
                                        center_obj=center_obj,
                                        scale=scale)
    contact_dist_tab = pd.merge(XY_contact_dist, Z_contact_dist, on=["object", "scale"])
    dist_tabs.append(contact_dist_tab)
indexes.clear()
combined_dist_tab = pd.concat(dist_tabs, ignore_index=True)
combined_dist_tab.insert(loc=0,column='image_name',value=test_img_name.stem)

Here, we can view the distribution metrics created by the above block of code. 

>NOTE: If include_contact_dist is False, the following code will produce an error as there will be no data to display.

In [ ]:
display(combined_dist_tab)

## DEFINE FUNCTIONS:
This section creates new functions that should run the contact metrics. These functions are based on the code from above.

### Make Dictionary
Creates a dictionary of all organelle segmentations with their keys assigned to their names

In [ ]:
def _make_dict(obj_names: list[str],                                        #Intakes list of object names
               obj_segs: list[np.ndarray]):                                 #Intakes list of object segmentations
    objs_labeled = {}                                                       #Initialize dictionary
    for idx, name in enumerate(obj_names):                                  #Loop across each organelle name
        if name == 'ER':                                                    #Proceed only for ER
            objs_labeled[name]=(obj_segs[idx]>0).astype(np.uint16)          #Ensures ER is labeled only as one object & sets it as key for its object segmentation
        else:                                                               #Proceed for other organelles
            objs_labeled[name]=obj_segs[idx]                                #Set the organelle name as the key for the corresponding object segmentation
    return objs_labeled                                                 #Return a dictionary of segmented objects with keys as the organelle name


### Make Contacting Function
Using the logic from the above 2-Way Contacts method and n-Way Contacts method, a new function capable of making contacts between n number of organelles is created.

In [ ]:
def _contacting(segs: dict[str], 
               iterated:dict[str],
               splitter:str="_") -> dict:
    conts = {} 
    for a in iterated:                                                                  #Iterates through preexisting dictionary of contact sites
        for b in segs:                                                                #Iterates through labeled organelles list (b)
            if(np.any(((iterated[a]>0)*(segs[b]>0))>0) and not (a==b) and not         #Proceeds if there is a contact between the lower order contact and the other organelle
               ((b+splitter in a)or(splitter+b in a)or(splitter+b+splitter in a)) and   #Proceeds if organelle is not already in lower order contact set
               (not inkeys(conts,(a+splitter+b),splitter=splitter))):                #Proceeds if contact between organelle and lower ordercontact set is not already made  
                conts[(a+splitter+b)]=label((iterated[a]*segs[b])>0)               #Adds new labeled contact
    return conts                                                                     #Returns contacts

### Make n-Way Contacts
Using the logic from the above n-Way contacts section, this _multi_contact function employs the new _contacting function to create the 2-way and n-Way contacts between organelles in addition to finding the highest order contacts.

In [ ]:
def _multi_contact(org_segs:dict[str:np.ndarray], 
                  organelles:list[str],
                  splitter:str="_",
                  redundancy:bool=True) -> dict:
    contacts=_contacting(org_segs, org_segs, splitter=splitter)     #Dictionary for ALL contacts, starts with 2nd Order                                     #Returns with dictionary of all levels of 

    iterated={}                            #Iterated Dictionary
    contact={}                             #Contact Dictionary
    HO_conts={}
    for n in (range(len(organelles)-1)): 
        iterated.clear()                   #Clears iterated dictionary
        contact.clear()                    #Clears contact dictionary
        num=n+3                            #Number of contacts in lower order number

    ##########################
    # Finding nLO Contacts
    ##########################
        for key in contacts:                                    #Iterates over every key in contacts
            if (len(key.split(splitter)) == (num-1)):           #Selects for last set of contacts
                iterated[key]=contacts[key]                     #Adds each key from last set of contacts to iterated

    ###########################
    # Making the n-Way Contact
    ###########################
        for c in iterated:                                                                  #Iterates through preexisting dictionary of contact sites
            for d in org_segs:                                                              #Iterates through labeled organelles list (b)
                if(np.any((iterated[c]*org_segs[d])>0) and not
                   ((d+splitter in c)or(splitter+d in c)or(splitter+d+splitter in c)) and   #Proceeds if organelle is not already in previous contact set
                   (not inkeys(contact,(c+splitter+d),splitter=splitter))):                 #Proceeds if contact between organelle and previous contact set is not already made  
                    contact[(c+splitter+d)]=label((iterated[c]*org_segs[d])>0)              #Adds new binary contact

    ################################################
    # Adding n-Way contacts to contacts Dictionary
    ################################################
        for key in contact:              #Iterates across the new contact sites 
            contacts[key]=(contact[key])   #Labels & assigns higher order contact to 

    ################################################
    # Dictionary of Non-Redundant Contacts
    ################################################
        if not redundancy:
            while len(iterated) != 0:
                key = list(iterated.keys())[0]
                ara = iterated[key]>0
                HO_conts[key] = ara
                for ki in contact.keys():
                    if (set(ki.split(splitter)).issuperset(key.split(splitter))):
                        HO_conts[key] = HO_conts[key]*(np.invert(watershed(image=(np.invert(ara)),                           #Watershed with the map of the nLO image
                                                                                  markers=contact[ki],                       #Watershed with the markers of the nO's superstr
                                                                                  mask=ara,                                  #Watershed unable to proceed beyond the nLO image
                                                                                  connectivity=np.ones((3, 3, 3), bool))>0)) #Watershed with 3D connectivity
                del iterated[key]
    if not redundancy:
        return contacts, HO_conts
    else:
        return contacts

### Get Contact Metrics
This function combines multiple steps seen in the above "Get Contact Metrics for Chosen n-Way Contact" section to produce a data table for the contact metrics and provides a True or False statement to enable or disable the production of a data table for the distribution metrics.

In [ ]:
def _get_contact_metrics_3D(orgs: str,
                        site: np.ndarray,
                        HO: np.ndarray,
                        organelle_segs: dict[str:np.ndarray],
                        mask: np.ndarray,
                        splitter: str="_",
                        scale: Union[tuple, None]=None,
                        include_dist:bool=False, 
                        dist_centering_obj: Union[np.ndarray, None]=None,
                        dist_num_bins: Union[int, None]=None,
                        dist_zernike_degrees: Union[int, None]=None,
                        dist_center_on: Union[bool, None]=None,
                        dist_keep_center_as_bin: Union[bool, None]=None) -> list:
    dist_tabs =[]
    labels = label(apply_mask(site, mask)).astype("int")    #Isolate to only contact sites found within the cell of interest
    para_labels = apply_mask((HO>0), mask).astype("int") * labels #copy labels found in labels to para_labels
    ##########################################
    ## CREATE LIST OF REGIONPROPS MEASUREMENTS
    ##########################################
    # start with LABEL
    properties = ["label"]

    # add position
    properties = properties + ["centroid", "bbox"]

    # add area
    properties = properties + ["area", "equivalent_diameter"] # "num_pixels", 

    # add shape measurements
    properties = properties + ["extent", "euler_number", "solidity", "axis_major_length", "slice"] # "feret_diameter_max", "axis_minor_length", 

    ##################
    ## RUN REGIONPROPS
    ##################
    props = regionprops_table(labels, intensity_image=None, properties=properties, extra_properties=None, spacing=scale)

    ##################################################################
    ## RUN SURFACE AREA FUNCTION SEPARATELY AND APPEND THE PROPS_TABLE
    ##################################################################
    surface_area_tab = pd.DataFrame(surface_area_from_props(labels, props, scale))

    ######################################################
    ## LIST WHICH ORGANELLES ARE INVOLVED IN THE CONTACT
    ######################################################    
    cont_inv = []                                               #initializes a variable to be used for creating the values for the contacts in the dictionary
                                                                  #   This variable is a list of the labels of each organelle involved in one contact site between those organelles
    involved = orgs.split(splitter)                             #creates list of all involved organelles in the contact
    indexes = dict.fromkeys(involved, [])                       #A dictionary of indexes of site involved in the contact
    indexes[orgs] = []                                           #   str = "contact" or "organelle", may have multiple different organelles
                                                                  #   list = contact or organelle number in image corresponding to the same contact in the other keys
                                                                #   a 2-way contact will have 3 keys, a 3-way contact will have 4 keys, etc
    redundancy = []
    for index, l in enumerate(props["label"]):
        cont_inv.clear()                                        #clears cont_inv variable of any labels from past contact site for new contact site
        present = para_labels[props["slice"][index]]
        present = present==l
        redundant = not np.any(present)
        redundancy.append(redundant)
        for org in involved:                                    #iterates across list of involved organelles
            volume = labels[props["slice"][index]]
            lorg = organelle_segs[org][props["slice"][index]]
            volume = volume==l
            lorg = lorg[volume]
            all_inv = np.unique(lorg[lorg>0]).tolist()
            if len(all_inv) != 1:                               #ensures only one label is involved in the contact site
                print(f"we have an error.  as-> {all_inv}")     #informs the console of any errors and the reasoing for it
            indexes[org].append(f"{all_inv[0]}")                #adds the label of the organelle involved in the contact to the organelle's key's list
            cont_inv.append(f"{all_inv[0]}")                    #adds the label of the organelle involved in the contact to the list of involved organelle labels
        indexes[orgs].append("_".join(cont_inv))                #adds the combination of all the organelle's labels involved in the contact to the contact key's list                                                
    ######################################################
    ## CREATE COMBINED DATAFRAME OF THE QUANTIFICATION
    ######################################################
    props_table = pd.DataFrame(props)
    props_table.drop(columns=['slice', 'label'], inplace=True)
    props_table.insert(0, 'label',value=indexes[orgs])
    props_table.insert(0, "object", orgs)
    props_table.rename(columns={"area": "volume"}, inplace=True)

    props_table.insert(11, "surface_area", surface_area_tab)
    props_table.insert(13, "SA_to_volume_ratio", 
    props_table["surface_area"].div(props_table["volume"]))
    props_table["redundant"] = redundancy
    
    if scale is not None:
        round_scale = (round(scale[0], 4), round(scale[1], 4), round(scale[2], 4))
        props_table.insert(loc=2, column="scale", value=f"{round_scale}")
    else: 
        props_table.insert(loc=2, column="scale", value=f"{tuple(np.ones(labels.ndim))}") 

    ######################################################
    ## optional: DISTRIBUTION OF CONTACTS MEASUREMENTS
    ######################################################
    if include_dist:
        XY_contact_dist, XY_bins, XY_wedges = get_XY_distribution(mask=mask, 
                                                                  obj=site,
                                                                  obj_name=orgs,
                                                                  centering_obj=dist_centering_obj,
                                                                  scale=scale,
                                                                  center_on=dist_center_on,
                                                                  keep_center_as_bin=dist_keep_center_as_bin,
                                                                  num_bins=dist_num_bins,
                                                                  zernike_degrees=dist_zernike_degrees)
       
        Z_contact_dist = get_Z_distribution(mask=mask,
                                            obj=site,
                                            obj_name=orgs,
                                            center_obj=dist_centering_obj,
                                            scale=scale)
        contact_dist_tab = pd.merge(XY_contact_dist, Z_contact_dist, on=["object", "scale"])
        dist_tabs.append(contact_dist_tab)
    indexes.clear()

    if include_dist:
        return props_table, dist_tabs
    else:
        return props_table

## OUTPUT:
Here, we will use the functions we created above to create the output from earlier that was made without using the functions while expanding on it to repeat across all of the contact sites.

In [ ]:
distance_tabs = []
contacts_tabs = []
labeled_dict = _make_dict(org_names, org_segs, also_binary=False)
ctcs = _multi_contact(labeled_dict, org_names, splitter=splitter)
if include_contact_dist:
    for orgs, site in ctcs.items():
        ctc_tab, dist_tab = _get_contact_metrics_3D(orgs=orgs,
                                                    site=site,
                                                    organelle_segs=labeled_dict,
                                                    mask=mask,
                                                    splitter=splitter,
                                                    scale=scale,
                                                    include_dist=include_contact_dist,
                                                    dist_centering_obj=region_segs[masks_file_name.index(dist_centering_obj)],
                                                    dist_num_bins=dist_num_bins,
                                                    dist_zernike_degrees=dist_zernike_degrees,
                                                    dist_center_on=dist_center_on,
                                                    dist_keep_center_as_bin=dist_keep_center_as_bin)
        for tabs in dist_tab:
            distance_tabs.append(tabs)
        contacts_tabs.append(ctc_tab)
        print(f"finished {orgs}")
else:
    for orgs, site in ctcs.items():
        ctc_tab = _get_contact_metrics_3D(orgs=orgs,
                                          site=site,
                                          mask=mask,
                                          splitter=splitter,
                                          scale=scale,
                                          include_dist=False)

The following block of code enables viewing of the contact metrics table.

In [ ]:
final_contact_tab = pd.concat(contacts_tabs, ignore_index=True)
final_contact_tab.insert(loc=0,column='image_name',value=test_img_name.stem)
display(final_contact_tab)

The following block of code enables viewing of the distribution metrics table.

In [ ]:
final_dist_tab = pd.concat(distance_tabs, ignore_index=True)
final_dist_tab.insert(loc=0,column='image_name',value=test_img_name.stem)
display(final_dist_tab)